<a href="https://colab.research.google.com/github/mamud1977/bootup/blob/main/NN_Recurrent_Stock%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf



In [16]:
data = pd.read_csv('/content/sample_data/GOOG.csv')

data.head(4)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,49.813286,51.835709,47.800831,49.982655,49.982655,44871300
1,2004-08-20,50.316402,54.336334,50.062355,53.952770,53.952770,22942800
2,2004-08-23,55.168217,56.528118,54.321388,54.495735,54.495735,18342800
3,2004-08-24,55.412300,55.591629,51.591621,52.239193,52.239193,15319700


In [23]:
data.max(), data.min()

(Date          2019-10-04
 Open              1274.0
 High          1289.27002
 Low          1266.295044
 Close        1287.579956
 Adj Close    1287.579956
 Volume          82541500
 dtype: object,
 Date         2004-08-19
 Open          49.409801
 High          50.680038
 Low           47.800831
 Close         49.818268
 Adj Close     49.818268
 Volume             7900
 dtype: object)

In [26]:
data[data['Date']> '2015-01-01']

,Date,Open,High,Low,Close,Adj Close,Volume
2611,2015-01-02,527.561584,529.815369,522.665039,523.373108,523.373108,1447500
2612,2015-01-05,521.827332,522.894409,511.655243,512.463013,512.463013,2059800
2613,2015-01-06,513.589966,514.761719,499.678131,500.585632,500.585632,2899900
2614,2015-01-07,505.611847,505.855164,498.281952,499.727997,499.727997,2065000
2615,2015-01-08,496.626526,502.101471,489.655640,501.303680,501.303680,3353500
...,...,...,...,...,...,...,...
3804,2019-09-30,1220.969971,1226.000000,1212.300049,1219.000000,1219.000000,1404100
3805,2019-10-01,1219.000000,1231.229980,1203.579956,1205.099976,1205.099976,1273500
3806,2019-10-02,1196.979980,1196.979980,1171.290039,1176.630005,1176.630005,1615100
3807,2019-10-03,1180.000000,1189.060059,1162.430054,1187.829956,1187.829956,1621200


In [44]:
train_data= data[data['Date'] < '2015-01-01']
test_data = data[data['Date'] >= '2015-01-01']

train_data.shape   ## (2611,7)


(2611, 7)

In [45]:
test_data.shape  ## (1198,7)

(1198, 7)

In [46]:
train_data.head(4)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,49.813286,51.835709,47.800831,49.982655,49.982655,44871300
1,2004-08-20,50.316402,54.336334,50.062355,53.952770,53.952770,22942800
2,2004-08-23,55.168217,56.528118,54.321388,54.495735,54.495735,18342800
3,2004-08-24,55.412300,55.591629,51.591621,52.239193,52.239193,15319700


In [48]:
train_data.drop(['Date','Adj Close'], axis=1, inplace=True)

KeyError: "['Date', 'Adj Close'] not found in axis"

In [49]:
train_data.head(4)

,Open,High,Low,Close,Volume
0,49.813286,51.835709,47.800831,49.982655,44871300
1,50.316402,54.336334,50.062355,53.952770,22942800
2,55.168217,56.528118,54.321388,54.495735,18342800
3,55.412300,55.591629,51.591621,52.239193,15319700


In [50]:
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
train_data[2:3]


array([[0.01025177, 0.01041574, 0.01166001, 0.00838273, 0.22215074]])

In [68]:
train_data.shape[1]

5

In [57]:
X_train = []
Y_train = []

for i in range(60, train_data.shape[0]):
  X_train.append(train_data[i-60:i])
  Y_train.append(train_data[i,0])

In [62]:
X_train[0:1]

[array([[7.18328756e-04, 2.05831180e-03, 0.00000000e+00, 2.94606433e-04,
         5.43577404e-01],
        [1.61403167e-03, 6.51205852e-03, 4.04404041e-03, 7.40965457e-03,
         2.77885613e-01],
        [1.02517709e-02, 1.04157429e-02, 1.16600116e-02, 8.38273019e-03,
         2.22150736e-01],
        [1.06863146e-02, 8.74780599e-03, 6.77866250e-03, 4.33866472e-03,
         1.85522018e-01],
        [5.11701597e-03, 5.55388206e-03, 7.05480056e-03, 5.34745434e-03,
         1.11762967e-01],
        [5.10814646e-03, 5.50952314e-03, 7.74958848e-03, 7.05256297e-03,
         8.62763771e-02],
        [7.90166616e-03, 6.10394616e-03, 8.66706896e-03, 5.48136033e-03,
         7.55243925e-02],
        [5.40079931e-03, 3.32700780e-03, 5.38907952e-03, 1.78545796e-03,
         6.31682127e-02],
        [2.75804167e-03, 1.74779047e-03, 5.52269863e-03, 2.10684272e-03,
         5.97732318e-02],
        [3.11277397e-03, 1.09126212e-03, 3.30470861e-03, 2.14255314e-04,
         1.11151095e-01],
        [0

In [72]:
X_train[0].shape

(60, 5)

In [63]:
Y_train[0:1]

[0.07630296751038479]

In [76]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers())


AttributeError: 'function' object has no attribute 'layers'